<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C13_MNIST_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
from google.colab import files
uploaded=files.upload()

Saving mnist_scaled.npz to mnist_scaled.npz


In [0]:
import numpy as np
mnist=np.load("mnist_scaled.npz")
X_train,y_train,X_test,y_test=[mnist[f] for f in mnist.files]

In [0]:
mean_val=np.mean(X_train,axis=0)
std_val=np.std(X_train)

In [0]:
X_train_centered=(X_train-mean_val)/std_val
X_test_centered=(X_test-mean_val)/std_val

In [0]:
print(X_train_centered.shape,y_train.shape)
print(X_test_centered.shape,y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [0]:
import tensorflow as tf
n_features=X_train_centered.shape[1]
n_classes=10
random_seed=123
np.random.seed(random_seed)


In [0]:
g=tf.Graph()
with g.as_default():
  tf.set_random_seed(random_seed)
  tf_x=tf.placeholder(dtype=tf.float32,shape=(None,n_features),name="tf_x")
  tf_y=tf.placeholder(dtype=tf.int32,shape=None,name="tf_y")
  y_onehot=tf.one_hot(indices=tf_y,depth=n_classes)
  h1=tf.layers.dense(inputs=tf_x,units=50,activation=tf.tanh,name="layer1")
  h2=tf.layers.dense(inputs=h1,units=50,activation=tf.tanh,name="layer2")
  logits=tf.layers.dense(inputs=h2,units=10,activation=None,name="layer3")
  predictions={
      "classes":tf.argmax(logits,axis=1,name="pedicted_classes"),
      "probabilities":tf.nn.softmax(logits,name="softmax_tensor")
  }

In [0]:
with g.as_default():
  cost=tf.losses.softmax_cross_entropy(onehot_labels=y_onehot,logits=logits)
  optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001)
  train_op=optimizer.minimize(loss=cost)
  init_op=tf.global_variables_initializer()

In [0]:
def create_batch_generator(X,y,batch_size=128,shuffle=False):
  X_copy=np.array(X)
  y_copy=np.array(y)
  if shuffle:
    data=np.column_stack((X_copy,y_copy))
    np.random.shuffle(data)
    X_copy=data[:,:-1]
    y_copy=data[:,-1].astype(int)
  
  for i in range(0,X.shape[0],batch_size):
    yield(X_copy[i:i+batch_size,:],y_copy[i:i+batch_size])
    

In [0]:
sess=tf.Session(graph=g)
sess.run(init_op)

In [0]:
for epochs in range(50):
  training_cost=[]
  batch_generator=create_batch_generator(X_train_centered,y_train,shuffle=True,batch_size=64)
  for batch_X,batch_y in batch_generator:
    feed={tf_x:batch_X,tf_y:batch_y}
    _,batch_cost=sess.run([train_op,cost],feed_dict=feed)
    training_cost.append(batch_cost)
  print("----Epochs----%2d----Average Training Loss:%.4f"%(epochs+1,np.mean(training_cost)))

----Epochs---- 1----Average Training Loss:1.5581
----Epochs---- 2----Average Training Loss:0.9492
----Epochs---- 3----Average Training Loss:0.7498
----Epochs---- 4----Average Training Loss:0.6385
----Epochs---- 5----Average Training Loss:0.5666
----Epochs---- 6----Average Training Loss:0.5159
----Epochs---- 7----Average Training Loss:0.4780
----Epochs---- 8----Average Training Loss:0.4486
----Epochs---- 9----Average Training Loss:0.4247
----Epochs----10----Average Training Loss:0.4052
----Epochs----11----Average Training Loss:0.3886
----Epochs----12----Average Training Loss:0.3742
----Epochs----13----Average Training Loss:0.3617
----Epochs----14----Average Training Loss:0.3507
----Epochs----15----Average Training Loss:0.3409
----Epochs----16----Average Training Loss:0.3319
----Epochs----17----Average Training Loss:0.3239
----Epochs----18----Average Training Loss:0.3165
----Epochs----19----Average Training Loss:0.3098
----Epochs----20----Average Training Loss:0.3036
----Epochs----21----

In [0]:
feed={tf_x:X_test_centered}
y_pred=sess.run(predictions['classes'],feed_dict=feed)
# print(y_pred.shape)
# print(y_test.shape)
print("Test Accuracy:%.3f%%"%(100*np.sum(y_pred==y_test)/y_test.shape[0]))

Test Accuracy:93.890%


In [0]:
import tensorflow as tf
import tensorflow.keras as keras
np.random.seed(123)
tf.set_random_seed(123)

In [0]:
y_train_onehot=keras.utils.to_categorical(y_train)

In [0]:
print("First 3 classes ",y_train[:3])
print("\nCatergorical 3 Classes \n",y_train_onehot[:3])

First 3 classes  [5 0 4]

Catergorical 3 Classes 
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [0]:
model=keras.models.Sequential()
model.add(keras.layers.Dense(kernel_initializer='glorot_uniform',units=50,input_dim=X_train_centered.shape[1],
                           bias_initializer='zeros',activation='tanh'))
model.add(keras.layers.Dense(units=50,input_dim=50,kernel_initializer='glorot_uniform',
                            bias_initializer='zeros',activation='tanh'))
model.add(keras.layers.Dense(units=y_train_onehot.shape[1],input_dim=50,kernel_initializer='glorot_uniform',
                            bias_initializer='zeros',activation='softmax'))

sgd_optimizer=keras.optimizers.SGD(lr=0.001,decay=1e-7,momentum=.9)
model.compile(optimizer=sgd_optimizer,loss='categorical_crossentropy')

In [0]:
history = model.fit(X_train_centered,y_train_onehot,batch_size=64,epochs=50,
                   verbose=1,
                   validation_split=.1)

Train on 54000 samples, validate on 6000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
54000/54000 [==============================] - 3s 53us/sample - loss: 0.7164 - val_loss: 0.3624
Epoch 2/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.3724 - val_loss: 0.2759
Epoch 3/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.3077 - val_loss: 0.2395
Epoch 4/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.2720 - val_loss: 0.2177
Epoch 5/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.2471 - val_loss: 0.2017
Epoch 6/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.2276 - val_loss: 0.1879
Epoch 7/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.2116 - val_loss: 0.1782
Epoch 8/50
54000/54000 [==============================] - 2s 39us/sample - loss: 0.1980 - val_loss: 0.1700
Epoch 9/50
54000/54000 [=======================

In [0]:
y_train_pred=model.predict_classes(X_train_centered,verbose=0)
print("Predicted 3 o/p ",y_train_pred[:3])

Predicted 3 o/p  [5 0 4]


In [0]:
correct_pred=np.sum(y_train_pred==y_train,axis=0)
print("Training Accuracy:",(correct_pred/y_train.shape[0])*100)

Training Accuracy: 98.87166666666667


In [0]:
y_test_pred=model.predict_classes(X_test_centered,verbose=0)
correct_pred_test=np.sum(y_test==y_test_pred,axis=0)
print("Test Accuracy:",(correct_pred_test/y_test.shape[0])*100)

Test Accuracy: 96.34
